In [29]:
from header import *


set_seed(33244233)
N = 14

model = nsp.model.UnitaryRiemanGenerator(D, dtype=torch.float64)
# model_sl = nsp.model.SlRiemanGenerator(D, dtype=torch.float64)
model._params.data[:] = torch.eye(model.D).view(-1)
# model_sl._params.data[:] = torch.eye(model.D).view(-1)


X = np.random.randn(D**2, D**2)
# X = np.random.randn(D, D)

X = (X + X.T)
# mask = X>=(1/(D**2))
# X[mask] = 0

loss_sl = nsp.loss.MES_SL(torch.Tensor(X), [D,D], inv = model_sl._inv)
loss = nsp.loss.MES(torch.Tensor(X), [D,D], inv = model._inv)


cg = RiemanUnitaryCG(model,loss, lr = 0.1)
cg_sl = RiemanSlCG(model,loss_sl )

# S, W = cg._riemannian_grad(model._params)

solver_cg = UnitarySymmTs(RiemanUnitaryCG, model, loss, lr = 0.1)
ret_cg = solver_cg.run(200, disable_message=False)

dtype changes from torch.float32 to torch.float64
target loss      : 36.7649192810
initial loss     : 57.0329045895




  0%|          | 0/200 [00:00<?, ?it/s]

No local minimum found
4.707100857875599e-19 3.948736411985009e-15 57.03290458950981 57.03290458950981
No local minimum found
2.786398699561864e-18 3.2868761081772386e-16 57.03290458950981 57.0329045895098
No local minimum found
-1.7437395858757766e-18 3.5111028410028663e-15 57.03290458950981 57.032904589509755
stack in local minimum --> break loop


In [27]:
loss(model.matrix()).backward()
cg._riemannian_grad(model._params)

dtype changes from torch.float32 to torch.float64


(tensor([[ 0.0000, -0.6752, -0.8750,  0.9680],
         [ 0.6752,  0.0000, -0.7107,  0.1720],
         [ 0.8750,  0.7107,  0.0000,  0.5287],
         [-0.9680, -0.1720, -0.5287,  0.0000]], dtype=torch.float64,
        grad_fn=<SubBackward0>),
 tensor([[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.]], dtype=torch.float64, grad_fn=<ViewBackward0>))

In [26]:
loss_sl(model_sl.matrix()).backward()
cg_sl._riemannian_grad(model_sl._params)

dtype changes from torch.float32 to torch.float64


(tensor([[-6.6022e-16, -2.7693e-01, -3.5885e-01,  3.9698e-01],
         [ 2.7693e-01, -8.9926e-16, -2.9146e-01,  7.0556e-02],
         [ 3.5885e-01,  2.9146e-01,  1.7075e-16,  2.1684e-01],
         [-3.9698e-01, -7.0556e-02, -2.1684e-01,  4.5532e-16]],
        dtype=torch.float64),
 tensor([[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.]], dtype=torch.float64))

In [24]:
model_sl._params.grad.view(4,4)

tensor([[ 6.6613e-16,  2.2905e-01,  1.5359e-01, -8.2214e-02],
        [-2.2905e-01,  4.5797e-16, -7.1891e-02, -4.3636e-01],
        [-1.5359e-01,  7.1891e-02,  2.7894e-15, -6.1658e-01],
        [ 8.2214e-02,  4.3636e-01,  6.1658e-01, -1.1102e-16]],
       dtype=torch.float64)

In [19]:
for _ in range(10):
    loss(model.matrix()).backward()
    loss(model2.matrix()).backward()
    cg2.step()
    cg.step()
    cg.zero_grad()
    cg2.zero_grad()

In [20]:
model.matrix()

tensor([[ 0.6798,  0.0555,  0.4857,  0.5467],
        [ 0.1244,  0.3583,  0.5885, -0.7140],
        [-0.1170, -0.8943,  0.4119, -0.1297],
        [-0.7132,  0.2621,  0.4980,  0.4178]], dtype=torch.float64,
       grad_fn=<ViewBackward0>)

In [21]:
model2.matrix()

tensor([[ 0.6798,  0.0555,  0.4857,  0.5467],
        [ 0.1244,  0.3583,  0.5885, -0.7140],
        [-0.1170, -0.8943,  0.4119, -0.1297],
        [-0.7132,  0.2621,  0.4980,  0.4178]], dtype=torch.float64,
       grad_fn=<ViewBackward0>)

In [15]:
lr = slcg._golden(W.data, S.data)

In [16]:
lr

1.275625102976006

In [8]:
loss(model.matrix()).backward()

In [10]:
model._params.grad

tensor([  1.2769, -12.3206,   7.5504,   7.5007,  -7.3384, -47.3078, -43.3932,
         28.8005,   1.4467,  27.3550,   8.5543, -16.6534, -12.1922, -75.3265,
        -72.0942,  45.8580], dtype=torch.float64)

In [38]:
model.matrix()

tensor([[ 1.2359, -0.7446,  0.8197,  1.7623],
        [-0.4031,  0.7149,  0.1083,  0.1054],
        [-0.3327, -0.3558, -0.2180,  0.1962],
        [-1.1024, -0.0299,  0.3727, -0.9463]], dtype=torch.float64,
       grad_fn=<ViewBackward0>)

In [6]:
from scipy.integrate import quad
import numpy as np
def integrand(x, a, b):
    return a*x**2 + b
a = 2
b = 1
I = quad(integrand, 0, 1, args=(a,b))

In [8]:
from scipy.integrate import quad
def integrand(t, n, x):
    return np.exp(-x*t) / t**n
def expint(n, x):
    return quad(integrand, 1, np.inf, args=(n, x))[0]
vec_expint = np.vectorize(expint)
import scipy.special as special
special.expn(3, np.arange(1.0,4.0,0.5))

array([0.10969197, 0.05673949, 0.03013338, 0.01629537, 0.00893065,
       0.00494538])

In [9]:
from scipy.integrate import quad, dblquad
def I(n):
    return dblquad(lambda t, x: np.exp(-x*t)/t**n, 0, np.inf, lambda x: 1, lambda x: np.inf)

In [10]:
print(I(4))

(0.2500000000043577, 1.2983033469368098e-08)
